In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import wandb
from wandb.keras import WandbCallback
from utils.ModelWrapper import ModelWrapper

In [ ]:
config_defaults = {
    'epochs': 30,
    'batch_size': 32,
    'learning_rate': 0.0001,
    'dropout': 0.5,
    'regularization': 0.0001,
}
wandb.init(config=config_defaults, project="deepfake-resnet", entity="dat550")
config = wandb.config

In [2]:
img_size = 128
batch_size = config.batch_size
data_dir = "./data/processed"

model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size)

In [3]:
class_names=os.listdir(f'{data_dir}/train')
class_names

['FAKE', 'REAL']

In [56]:
model_file = f'models/resnet_{wandb.run.name}_model.h5'

pretrained_model = ResNet152(include_top=False,
                   input_shape=(img_size, img_size, 3),
                   pooling='max',
                   classes=2,
                   weights='imagenet'
                   )

model_layers = [
    pretrained_model,
    Dense(units = 512, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(config.regularization), bias_regularizer=tf.keras.regularizers.L2(config.regularization)),
    Dropout(config.dropout),
    Dense(units = 128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(config.regularization), bias_regularizer=tf.keras.regularizers.L2(config.regularization)),
    Dense(units = 1, activation = 'sigmoid'),
]

model_wrapper.create_model(model_file, model_layers, config)
model_wrapper.model.compile(optimizer=Adam(learning_rate=config.learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model_wrapper.model.summary()


In [ ]:
history = model_wrapper.fit()
print(history.history)

### Evaluate

In [ ]:
model_wrapper.evaluate_model(model_wrapper.model)

In [ ]:
model_wrapper.export_to_png()